<a href="https://colab.research.google.com/github/comet-toolkit/comet_training/blob/main/LPS_training_exercise3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **LPS training session: Exercise 3**

### **Introduction**

Within this exercise, we will run through processing and comparing a bottom-of-atmosphere (BOA) Landsat-9 (L9) matchup with HYPERNETS data over the Gobabeb site in Namibia. 
This matchup was taken from the list of matchups identified in De Vis et al. (2024; https://doi.org/10.3389/frsen.2024.1323998).

We will assume a starting point where the matchup has already been identified and the files have already been downloaded. The matchup is in June 2022.
As part of this notebook, we will read in the HYPERNETS data, mask poor-quality data, and process the data to be comparible with L9 (e.g. spectral band integration). The bias will then be calculated and plotted. Your job will be to add uncertainty propagation to this process. These uncertainties will come from the product files themselves, and can be propagated throughout the processing chain using the CoMet toolkit.

We first install the matheo package, which can be used for band integration.

In [ ]:
!pip install matheo==0.1.3

We also import all the relevant python packages we use in this training:

In [ ]:
import xarray as xr
import numpy as np
from punpy import MeasurementFunction, MCPropagation
from matheo.band_integration import band_integration
from obsarray.templater.dataset_util import DatasetUtil
import matplotlib.pyplot as plt

Next, we clone the hypernets_training repository, which contains all the datasets used in this training.

In [ ]:
!git clone https://github.com/comet-toolkit/comet_training.git

**reading in satellite data**

In [ ]:
ds_refl_L9_hyp = xr.load_dataset("comet_training/example_L9_20220606.nc")     # ROI centred on HYPERNETS

From these we can determine the reflectance in each band from the mean over the ROI, and its uncertainty (combination of noise and spatial variability) from the standard deviation between pixels:

In [ ]:
bands_L9 = ["B1", "B2", "B3", "B4", "B5", "B6"]
wav_L9 = [442.98244284, 482.58889933, 561.33224557, 654.60554515, 864.5708545, 1609.09056245]

refl_L9_hyp = np.array([np.mean(ds_refl_L9_hyp[band].values) for band in bands_L9])
u_refl_L9_hyp = np.array([np.std(ds_refl_L9_hyp[band].values) for band in bands_L9])

**reading in HYPERNETS data**

Then, we open the HYPERNETS L2A dataset (surface reflectance without site-specific quality checks). These data were downloaded from the LANDHYPERNET data portal (https://landhypernet.org.uk). 

In [ ]:
ds_HYP = xr.open_dataset("comet_training/HYPERNETS_L_GHNA_L2A_REF_20220606T0900_20231226T1435_v2.0.nc")  # read digital effects table

Next, to illustrate the LANDHYPERNET data, we make a spectral plot for a specific viewing geometry (), and a polar polot for a specific wavelength (550nm).

In [ ]:
def plot_illustrative_hypernets(dataset, wavelength, series =22):
        # first plot the spectrum for an example geometry:

        fig = plt.figure(figsize=(15,6))
        ax = plt.subplot(121)
        ax.plot(dataset["wavelength"].values, dataset["reflectance"].values[:,series], 
                label = "HYPERNETS spectrum at vza=%s, vaa=%s"%(dataset["viewing_zenith_angle"].values[series],dataset["viewing_azimuth_angle"].values[series]))
        hyp_unc = dataset["reflectance"].values[:,series] /100 * np.sqrt(dataset["u_rel_random_reflectance"].values[:,series]**2+dataset["u_rel_systematic_reflectance"].values[:,series]**2)
        ax.fill_between(
          dataset["wavelength"].values, dataset["reflectance"].values[:,series] - hyp_unc, dataset["reflectance"].values[:,series] + hyp_unc, alpha=0.2, color="b"
        )
        ax.legend()
        ax.set_ylim(0,0.6)


        #next make a polar plot for given wavelength


        saa = np.mean(dataset.solar_azimuth_angle.values % 360)
        sza = np.mean(dataset.solar_zenith_angle.values)
        vaa = dataset.viewing_azimuth_angle.values % 360

        vza = dataset.viewing_zenith_angle.values
        refl = dataset.reflectance.values

        vaa_grid = np.arange(8, 368, 15)
        vza_grid = np.array([0, 5, 10, 20, 30, 40, 50, 60])
        raa_grid = vaa_grid - saa

        id_wav = np.argmin(np.abs(wavelength - dataset.wavelength.values))

        vaa_mesh, vza_mesh = np.meshgrid(np.radians(vaa_grid), vza_grid)

        refl_2d = np.zeros((len(vaa_grid), len(vza_grid)))
        for i in range(len(vaa_grid)):
            for ii in range(len(vza_grid)):
                id_series = np.where(
                    (
                        np.abs(vaa - vaa_grid[i])
                        < 2
                    )
                    & (
                        np.abs(vza - vza_grid[ii])
                        < 2
                    )
                )[0]
                if len(id_series) == 1:
                    refl_2d[i, ii] = np.abs(refl[id_wav, id_series])
                elif len(id_series) > 1:
                    print(
                        "There are multiple series that match the same vaa (%s) and vza (%s) "
                        "within a tolerance of %s and %s degrees respectively."
                        % (
                            vaa_grid[i],
                            vza_grid[ii],
                            2,
                            2,
                        )
                    )
                    refl_2d[i, ii] = np.mean(np.abs(refl[id_wav, id_series]))
        refl_2d[refl_2d == 0] = np.nan
        ax2 = plt.subplot(122, projection='polar')
        ax2.set_theta_direction(-1)
        ax2.set_theta_offset(np.pi / 2.0)
        im = ax2.pcolormesh(
            vaa_mesh,
            vza_mesh,
            refl_2d.T,
            shading="auto",
            cmap=plt.get_cmap("jet"),
        )

        ax2.plot(np.radians(saa), sza, color="k", ls="none", marker="o")

        cbar = fig.colorbar(im)
        cbar.set_label("reflectance at %s nm" % wavelength, rotation=270, labelpad=15)

        plt.show()
        plt.close(fig)

plot_illustrative_hypernets(ds_HYP, 550, series=18)

Have a look at the data available in this dataset. As shown in the dimensions of the dataset, the LANDHYPERNETS products samples at 1551 wavelengths and 44 different geometries. 

In [ ]:
ds_HYP

**masking HYPERNETS data with obsarray (feel free to skip if not interested in flagging)**

In the L2A dataset, there is a quality flag variable, which contains quality flags indicating warnings about the data quality. For further information on how to handle these flags, we refer to the following jupyter notebook: https://colab.research.google.com/github/comet-toolkit/comet_training/blob/main/hypernets_surface_reflectance.ipynb . Below, we list the quality flags set in the current dataset (one list for each geometry).

In [ ]:
print([DatasetUtil.get_set_flags(flag) for flag in ds_HYP["quality_flag"]])

The data does have a few warnings (especially the missing series - which indicates at least one geometry from the standard sequence is not present in the data), but none of these are an issue for the current usecase. Some other products do contain more significant quality flags, for which we recommend not using this data for Cal/Val if the flag is raised as highlighted below. We next remove all data that has these "bad" quality flags set. In the near future, when the data is released publicly, the L2B data will be released, which will already have all these data with "bad" quality flags omitted.

In [ ]:

bad_flags = [
    "pt_ref_invalid",
    "half_of_scans_masked",
    "not_enough_dark_scans",
    "not_enough_rad_scans",
    "not_enough_irr_scans",
    "no_clear_sky_irradiance",
    "variable_irradiance",
    "half_of_uncertainties_too_big",
    "discontinuity_VNIR_SWIR",
    "single_irradiance_used",
]
flagged = DatasetUtil.get_flags_mask_or(ds_HYP["quality_flag"], bad_flags)
id_series_valid = np.where(~flagged)[0]
ds_HYP = ds_HYP.isel(series=id_series_valid)

**extracting relevant data from the files**

We note that for the dataset being used here, this does not remove any data. 
Once we have removed the flagged data, we select the geometry (i.e. series dimension) which most closely matches the satellite viewing angles (which were taken from the satellite data at the region of interest).

In [ ]:
vza_L9 = 0.7
#vaa_L9 = 38.1
vzadiff = ds_HYP["viewing_zenith_angle"].values - vza_L9
#vaadiff = np.abs(ds_HYP["viewing_azimuth_angle"].values - vaa_L9 % 360)
angledif_series = np.sqrt(vzadiff**2) # + vaadiff**2)
id_series = np.where(angledif_series < 5)[0]
ds_HYP = ds_HYP.isel(series=id_series)

We do not select on viewing azimuth angle here, as for nadir measurements there is only a single measurement.
We do here give a word of caution around using the LANDHYPERNET nadir measurements as these are prone to shadowing. 
Measurements affected by shadowing will be removed in the L2B files, but are still present in L2A.

The most relevant variables from the xarray dataset are extracted as numpy arrays:

In [ ]:
wav_HYP_full = ds_HYP["wavelength"].values
refl_HYP_full = np.mean(ds_HYP["reflectance"].values,axis=1)
u_ran_refl_HYP_full = refl_HYP_full * np.mean(ds_HYP["u_rel_random_reflectance"].values,axis=1) / 100
u_sys_refl_HYP_full = refl_HYP_full * np.mean(ds_HYP["u_rel_systematic_reflectance"].values,axis=1) / 100
u_refl_HYP_full = np.sqrt(u_ran_refl_HYP_full**2+u_sys_refl_HYP_full**2)  # add random and systematic uncertainties for later plotting
err_corr_sys_refl = ds_HYP["err_corr_systematic_reflectance"].values

We note that spectrally, the systematic component of the HYPERNETS data is not 100% correlated (but it is fully correlated between repeated measurements/different geometries), but instead an spectral error correlation matrix is provided with the data. 

In [ ]:
plt.imshow(err_corr_sys_refl)
plt.show()
plt.close() 

**band integration with L9 SRF**

The LANDHYPERNET data has a Gaussian spectral response function (SRF) with a width of 3nm for the VNIR sensor (<1000nm) and 10nm for the SWIR sensor (>1000nm). In order to be comparable with the satellite data, we need to spectrally integrate this data to the SRF of the L9 OLI sensor. We here provide a function that performs this band integration using the matheo tool, and returns the first six L9 bands, which we are including in this comparison. 

In [ ]:

def band_integrate_L9(reflectance, wavelength):
    """
    Function to perform L9 band integration on reflectance

    :param reflectance: reflectance spectrum
    :param wavelength: wavelengths
    """
    refl_band, band_centres = band_integration.spectral_band_int_sensor(
        d=reflectance,
        wl=wavelength,
        platform_name="Landsat-8",  # the pyspectral library does not contain the L9 SRF, so we use L8 instead which is very similar.
        sensor_name="OLI",
        u_d=None,
    )
    return refl_band[:6]

We use this function to perform the band integration:

In [ ]:
refl_HYP = band_integrate_L9(refl_HYP_full,wav_HYP_full)


**calculating bias**

The HYPERNETS data are now at the same spectral scale as the L9 data. All the data is also near simultaneous, in a similar viewing geometry and over a homogeneous surface. So the data is now sufficiently consistent to perform the comparison. We calculate the bias between the satellite and reference as follows:

In [ ]:
bias_hyp = ((refl_L9_hyp / refl_HYP) - 1) * 100

We then also define some plotting functions with and without uncertainties (the later is for you to use later):

In [ ]:
def plot_no_uncertainties(
    sat: str,
    date: str,
    sat_wav: np.ndarray,
    sat_refl: np.ndarray,
    hyp_wav: np.ndarray,
    hyp_refl: np.ndarray,
    bias: np.ndarray,
    wavs_band: np.ndarray,
    reflectance_band: np.ndarray,
    vza: float,
):
    """
    Function to plot matchup results
    
    :param sat: satellite name
    :param date: date of matchup
    :param sat_wav: satellite wavelength
    :param sat_refl: satellite reflectance 
    :param hyp_wav: hypernets (full resolution) wavelength
    :param hyp_refl: hypernets reflectance
    :param bias: hypernets bias (for satellite bands)
    :param wavs_band: band-integrated hypernets wavelength
    :param reflectance_band: band-integrated hypernets reflectance
    :param vza: satellite viewing zenith angle
    :return: 
    """
    plt.figure(figsize=(20, 12))
    plt.subplot(2, 1, 1)
    plt.scatter(
        wavs_band,
        reflectance_band,
        marker="o",
        color="m",
        label="HYPERNETS for satellite bands",
    )
    plt.scatter(
        sat_wav, sat_refl, marker="o", color="g", label=sat
    )
    
    plt.plot(hyp_wav, hyp_refl, "-b", label="HYPERNETS full-resolution model")
    if sat == "Landsat-8" or sat == "Landsat-9":
        plt.title(
            "Landsat-8/9 (vza=%.1f) vs HYPERNETS TOA Comparison at %s" % (vza, date),
            fontsize=20,
        )
    else:
        plt.title(
            "%s (vza=%.1f) vs HYPERNETS Comparison at %s" % (sat, vza, date),
            fontsize=20,
        )
    plt.ylabel("Reflectance", fontsize=20)
    plt.xlim(380, 1700)
    plt.ylim(0.0, 0.6)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.legend(loc=2, numpoints=1, scatterpoints=1, facecolor="white")

    plt.subplot(2, 1, 2)
    plt.scatter(
        sat_wav,
        bias,
        marker="o",
        color="blue",
        label="HYPERNETS-%s bias" % sat,
    )
    plt.axhline(y=0, color="r", linestyle="--")
    plt.ylabel("Relative Difference (%)", fontsize=20)
    plt.xlabel("Wavelength (nm)", fontsize=20)
    plt.xlim(380, 1700)
    plt.ylim(-10, 10)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.legend()
    # plt.legend(loc=2, numpoints=1, scatterpoints=1, facecolor='white')
    plt.show()
    plt.close()

def plot_with_uncertainties(
    sat: str,
    date: str,
    sat_wav: np.ndarray,
    sat_refl: np.ndarray,
    sat_unc: np.ndarray,
    hyp_wav: np.ndarray,
    hyp_refl: np.ndarray,
    hyp_unc: np.ndarray,
    bias: np.ndarray,
    bias_unc: np.ndarray,
    wavs_band: np.ndarray,
    reflectance_band: np.ndarray,
    reflectance_band_unc: np.ndarray,
    vza: float,
):
    """
    Function to plot matchup results
    
    :param sat: satellite name
    :param date: date of matchup
    :param sat_wav: satellite wavelength
    :param sat_refl: satellite reflectance 
    :param sat_unc: satellite reflectance uncertainty
    :param hyp_wav: hypernets (full resolution) wavelength
    :param hyp_refl: hypernets reflectance
    :param hyp_unc: hypernets reflectance uncertainty
    :param bias: hypernets bias (for satellite bands)
    :param bias_unc: hypernets bias uncertainty (for satellite bands)
    :param wavs_band: band-integrated hypernets wavelength
    :param reflectance_band: band-integrated hypernets reflectance
    :param reflectance_band_unc: band-integrated hypernets reflectance uncertainty
    :param vza: satellite viewing zenith angle
    :return: 
    """
    plt.figure(figsize=(20, 12))
    plt.subplot(2, 1, 1)
    plt.errorbar(
        wavs_band,
        reflectance_band,
        yerr=reflectance_band_unc,
        fmt="o",
        ls="none",
        ms=10,
        color="m",
        label="HYPERNETS for satellite bands",
    )
    plt.errorbar(
        sat_wav, sat_refl, yerr=sat_unc, fmt="o", ls="none", ms=10, color="g", label=sat
    )
    plt.fill_between(
        hyp_wav, hyp_refl - hyp_unc, hyp_refl + hyp_unc, alpha=0.3, color="b"
    )
    plt.plot(hyp_wav, hyp_refl, "-b", label="HYPERNETS full-resolution model")
    if sat == "Landsat-8" or sat == "Landsat-9":
        plt.title(
            "Landsat-8/9 (vza=%.1f) vs HYPERNETS TOA Comparison at %s" % (vza, date),
            fontsize=20,
        )
    else:
        plt.title(
            "%s (vza=%.1f) vs HYPERNETS Comparison at %s" % (sat, vza, date),
            fontsize=20,
        )
    plt.ylabel("Reflectance", fontsize=20)
    plt.xlim(380, 1700)
    plt.ylim(0.0, 0.6)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.legend(loc=2, numpoints=1, scatterpoints=1, facecolor="white")

    plt.subplot(2, 1, 2)
    plt.errorbar(
        sat_wav,
        bias,
        yerr=bias_unc,
        fmt="o",
        mfc="blue",
        ls="none",
        ms=15,
        capsize=3,
        label="HYPERNETS-%s bias" % sat,
    )
    plt.axhline(y=0, color="r", linestyle="--")
    plt.ylabel("Relative Difference (%)", fontsize=20)
    plt.xlabel("Wavelength (nm)", fontsize=20)
    plt.xlim(380, 1700)
    plt.ylim(-10, 10)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.legend()
    # plt.legend(loc=2, numpoints=1, scatterpoints=1, facecolor='white')
    plt.show()
    plt.close()

and then finally make the plot without uncertainties:

In [ ]:
plot_no_uncertainties("LANDSAT 9", "2022-06-06", wav_L9, refl_L9_hyp, wav_HYP_full, refl_HYP_full, bias_hyp, wav_L9, refl_HYP, vza_L9)


**Here the real exercise begins**

Next, it is your job to propagate uncertainties through the spectral band integration of the HYPERNETS data, and through the bias calculation. Propagate the HYPERNETS random uncertainties and systematic uncertainties (and custom error correlation matrix) and combine these two components quadratically.

In [ ]:
prop = MCPropagation(50, parallel_cores=1)
u_ran_refl_HYP = prop.propagate_standard(band_integrate_L9,[refl_HYP_full,wav_HYP_full],[u_ran_refl_HYP_full,None],["rand",None])
u_sys_refl_HYP = prop.propagate_standard(band_integrate_L9,[refl_HYP_full,wav_HYP_full],[u_sys_refl_HYP_full,None],[ds_HYP["err_corr_systematic_reflectance"].values,None])
u_refl_HYP = np.sqrt(u_ran_refl_HYP**2+u_sys_refl_HYP**2)


Compare the band integrated uncertainties to the full resolution uncertainties, and see how they are affected differently by the process. 

In [ ]:
plt.plot(wav_HYP_full,u_ran_refl_HYP_full,"red",label="random uncertainty full resolution")
plt.plot(wav_HYP_full,u_sys_refl_HYP_full,"blue",label="systematic uncertainty full resolution")
plt.plot(wav_L9,u_ran_refl_HYP,"ro",label="random uncertainty band integrated")
plt.plot(wav_L9,u_sys_refl_HYP,"bo",label="systematic uncertainty band integrated")
plt.yscale("log") 
plt.legend()
plt.show()
plt.close()  

**Question 1:** Which of the uncertainty components is most reduced by the spectral band integration?

Next, calculate the uncertainties on the bias:

In [ ]:
# Note that this can perfectly be done using punpy, by just wrapping the bias calculation in a measurement function
# However given it is easy to calculate analytically, and this is faster than the MC propagation, we here just add the relative uncertainties in qaudrature:

u_bias_hyp = np.sqrt((u_refl_L9_hyp / refl_L9_hyp) ** 2 + (u_refl_HYP / refl_HYP) ** 2) * 100

Finally, make the bias plot with uncertainties, using the function provided previously. 

In [ ]:
plot_with_uncertainties("LANDSAT 9", "2022-06-06", wav_L9, refl_L9_hyp, u_refl_L9_hyp, wav_HYP_full, refl_HYP_full, u_refl_HYP_full, bias_hyp, u_bias_hyp, wav_L9, refl_HYP, u_refl_HYP, vza_L9)

**Question 2:** Within the shown examples, we have propagated some uncertainty components, but there are many others missing. Could you think of a few uncertainty components that are missing when comparing the in-situ data to satellites?